In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import seaborn as sns
import datetime as dt
import os
import json
from datetime import datetime, timedelta
import pandas as pd
import seaborn as sb
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf

In [2]:
sns.set_style("whitegrid")
default_figsize = (10, 5)
color1 = "#3c3c3d"
color2 = "#c99d66"
color3 = "#ecf0f1"
color4 = "#14044d"
color5 = "#48cbd9"
color6 = "#716b94"
color7 = "#30ba96"

In [3]:
## Check for stationarity
from statsmodels.tsa.stattools import adfuller

def is_stationary(timeseries):
    """
    Determines if a time series is stationary using the Augmented Dickey-Fuller (ADF) test.

    Parameters:
    timeseries (pandas.Series): The time series to test for stationarity.
    significance_level (float): The significance level used to determine the critical value for the test.

    Returns:
    bool: True if the time series is stationary, False otherwise.
    """

    # Perform ADF test
    adf_result = adfuller(timeseries)

    # Get test statistic and critical values
    test_statistic = adf_result[0]
    critical_values = adf_result[4]
    print("Result of Dickey-Fuller test:")
    df_output = pd.Series(adf_result[0:4], index=["Test Statistic", "p-value", "#Lags Used", "Number of Observations Used"])
    
    for key, value in adf_result[4].items():
        df_output["Critical Value (%s)"%key] = value
    print(df_output)
        
        
    # Compare test statistic to critical values to determine stationarity
    if test_statistic < critical_values['5%']:  ## 5% significance level
        return True
    else:
        return False
    


In [4]:
df = pd.read_csv("training_dataset.csv", index_col="date", parse_dates=True)

In [5]:
df.columns[df.isnull().any()].tolist()

['smi', 'smi_signal', 'ma_26', 'ema_26', 'upper_band', 'lower_band']

In [6]:
df[df.columns[df.isnull().any()].tolist()] = df[df.columns[df.isnull().any()].tolist()].fillna(method="bfill")

In [7]:
df.index.freq = "D"

In [8]:
df.head()

,month,eth_open,eth_high,eth_low,eth_close,eth_volume,eth_qav,eth_num_trades,eth_taker_base_vol,eth_taker_quote_vol,...,smi,smi_signal,ma_12,ma_26,ema_12,ema_26,obv,mfi,upper_band,lower_band
date,,,,,,,,,,,,,,,,,,,,,
2017-09-01,2017-09-01,386.44,394.39,383.18,388.46,4198.68434,1.635228e+06,4124.0,1564.05531,6.096309e+05,...,-12.212548,-21.164389,346.524167,326.348077,346.524167,326.348077,23109.64307,88.346876,396.369992,266.133008
2017-09-02,2017-09-01,388.58,394.32,320.08,343.14,8240.32724,2.904008e+06,8217.0,2460.66776,8.684873e+05,...,-12.212548,-21.164389,348.178333,326.348077,348.178333,326.348077,14869.31583,80.426793,396.369992,266.133008
2017-09-03,2017-09-01,343.56,367.28,320.08,341.77,5253.62211,1.801703e+06,6153.0,2240.91753,7.679383e+05,...,-12.212548,-21.164389,350.842500,326.348077,350.842500,326.348077,9615.69372,74.419284,396.369992,266.133008
2017-09-04,2017-09-01,339.29,344.55,267.94,283.77,6737.81061,2.002446e+06,7403.0,2954.16805,8.770111e+05,...,-26.876901,-21.164389,348.178333,326.348077,348.178333,326.348077,2877.88311,67.246410,396.369992,266.133008
2017-09-05,2017-09-01,286.81,321.52,235.55,312.09,8344.41924,2.397491e+06,8999.0,4312.97282,1.251959e+06,...,-24.403719,-21.164389,347.234167,326.348077,347.234167,326.348077,11222.30235,66.412993,396.369992,266.133008


In [11]:
df.head()

,month,eth_open,eth_high,eth_low,eth_close,eth_volume,eth_qav,eth_num_trades,eth_taker_base_vol,eth_taker_quote_vol,...,smi,smi_signal,ma_12,ma_26,ema_12,ema_26,obv,mfi,upper_band,lower_band
date,,,,,,,,,,,,,,,,,,,,,
2017-09-01,2017-09-01,386.44,394.39,383.18,388.46,4198.68434,1.635228e+06,4124.0,1564.05531,6.096309e+05,...,-12.212548,-21.164389,346.524167,326.348077,346.524167,326.348077,23109.64307,88.346876,396.369992,266.133008
2017-09-02,2017-09-01,388.58,394.32,320.08,343.14,8240.32724,2.904008e+06,8217.0,2460.66776,8.684873e+05,...,-12.212548,-21.164389,348.178333,326.348077,348.178333,326.348077,14869.31583,80.426793,396.369992,266.133008
2017-09-03,2017-09-01,343.56,367.28,320.08,341.77,5253.62211,1.801703e+06,6153.0,2240.91753,7.679383e+05,...,-12.212548,-21.164389,350.842500,326.348077,350.842500,326.348077,9615.69372,74.419284,396.369992,266.133008
2017-09-04,2017-09-01,339.29,344.55,267.94,283.77,6737.81061,2.002446e+06,7403.0,2954.16805,8.770111e+05,...,-26.876901,-21.164389,348.178333,326.348077,348.178333,326.348077,2877.88311,67.246410,396.369992,266.133008
2017-09-05,2017-09-01,286.81,321.52,235.55,312.09,8344.41924,2.397491e+06,8999.0,4312.97282,1.251959e+06,...,-24.403719,-21.164389,347.234167,326.348077,347.234167,326.348077,11222.30235,66.412993,396.369992,266.133008


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2014 entries, 2017-09-01 to 2023-03-07
Freq: D
Data columns (total 44 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   month                     2014 non-null   object 
 1   eth_open                  2014 non-null   float64
 2   eth_high                  2014 non-null   float64
 3   eth_low                   2014 non-null   float64
 4   eth_close                 2014 non-null   float64
 5   eth_volume                2014 non-null   float64
 6   eth_qav                   2014 non-null   float64
 7   eth_num_trades            2014 non-null   float64
 8   eth_taker_base_vol        2014 non-null   float64
 9   eth_taker_quote_vol       2014 non-null   float64
 10  btc_close                 2014 non-null   float64
 11  btc_volume                2014 non-null   float64
 12  btc_num_trades            2014 non-null   float64
 13  fed_eff_funds_rate_close  2014 non-nu

In [13]:
df.shape

(2014, 44)

# Basic

- Ethereum related data: eth_close, eth_volume, eth_num_trades

In [10]:
df[["eth_open","eth_close","eth_low", "eth_high","eth_volume","eth_num_trades", "eth_qav", "eth_taker_base_vol", "eth_taker_quote_vol"]].head()

,eth_open,eth_close,eth_low,eth_high,eth_volume,eth_num_trades,eth_qav,eth_taker_base_vol,eth_taker_quote_vol
date,,,,,,,,,
2017-09-01,386.44,388.46,383.18,394.39,4198.68434,4124.0,1.635228e+06,1564.05531,6.096309e+05
2017-09-02,388.58,343.14,320.08,394.32,8240.32724,8217.0,2.904008e+06,2460.66776,8.684873e+05
2017-09-03,343.56,341.77,320.08,367.28,5253.62211,6153.0,1.801703e+06,2240.91753,7.679383e+05
2017-09-04,339.29,283.77,267.94,344.55,6737.81061,7403.0,2.002446e+06,2954.16805,8.770111e+05
2017-09-05,286.81,312.09,235.55,321.52,8344.41924,8999.0,2.397491e+06,4312.97282,1.251959e+06


In [ ]:
df.drop(columns=["eth_open", "eth_high", "eth_low", "month", "eth_qav", "eth_taker_base_vol", "eth_taker_quote_vol"], inplace=True)

In [ ]:
basic = df[['eth_close','eth_volume','eth_num_trades']]

In [ ]:
basic.head()

In [ ]:
sns.heatmap(np.log(basic).diff().dropna().corr(), annot=True, cmap="Blues"); ##Percent Change correlation

In [ ]:
## Close and Volume
fig, ax = plt.subplots(figsize=default_figsize)
font = {'fontname':'Times New Roman', 'size': 13, 'weight' : 'bold'}


ax.set_title('ETH záró árfolyama és kereskedelmi volumene (USDT)', **font)
ax.set_xlabel('dátum', **font)
line1 = ax.plot(basic.index, basic.eth_close, color6, label="ETH napi záró árfolyama ($)")
ax.set_ylabel('ETH napi záró árfolyama ($)', **font)
ax.yaxis.set_major_formatter("${x:,.2f}")
ax.tick_params(axis="y", labelcolor=color6)

ax2 = ax.twinx()

line2 = ax2.plot(basic.index, basic.eth_volume, color2, label="ETH keresedelmi volumene ($)", alpha=0.8)
ax2.set_ylabel('ETH keresedelmi volumene ($)', **font)
fmt = '$%.0fM'
ticks = plt.FixedLocator([0, 2e6, 4e6, 6e6, 8e6, 10e6])
ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: fmt % (x * 1e-6)))
ax2.yaxis.set_major_locator(ticks)
ax2.tick_params(axis="y", labelcolor=color2)

ax2.spines['left'].set_color(color6)
ax2.spines['right'].set_color(color2)

lines = line1 + line2
labels = [l.get_label() for l in lines]
ax.legend(lines, labels, loc=0)

##MIN MAX POINT for CLOSE
# Find the minimum and maximum values
y_min = basic['eth_close'].min()
y_max = basic['eth_close'].max()

# Find the indices of the minimum and maximum values
min_idx = basic['eth_close'].argmin()
max_idx = basic['eth_close'].argmax()

# Add dots for the minimum and maximum values
ax.scatter(basic.index[min_idx], y_min, color=color6, s=15, zorder=3)
ax.scatter(basic.index[max_idx], y_max, color=color6, s=15, zorder=3)

# Add text labels for the minimum and maximum values
ax.text(basic.index[min_idx]+pd.Timedelta(days=10), y_min-200, f"Min: ${y_min:.2f} ({basic.index[min_idx].strftime('%d/%m/%y')}) ", fontsize=10, weight="bold", color=color6)
ax.text(basic.index[max_idx]+pd.Timedelta(days=15), y_max-200, f"Max: ${y_max:.2f} \n ({basic.index[max_idx].strftime('%d/%m/%y')}) ", fontsize=10, weight="bold", color=color6)

##MIN MAX POINT for VOLUME
# Find the minimum and maximum values
y_min = basic['eth_volume'].min()
y_max = basic['eth_volume'].max()

# Find the indices of the minimum and maximum values
min_idx = basic['eth_volume'].argmin()
max_idx = basic['eth_volume'].argmax()

# Add dots for the minimum and maximum values
ax2.scatter(basic.index[min_idx], y_min, color=color2, s=15, zorder=3)
ax2.scatter(basic.index[max_idx], y_max, color=color2, s=15, zorder=3)

# Add text labels for the minimum and maximum values
ax2.text(basic.index[min_idx]-pd.Timedelta(days=10), y_min-250000, f"Min: ${y_min:.2f} ({basic.index[min_idx].strftime('%d/%m/%y')})", fontsize=10, weight="bold", color=color2)
ax2.text(basic.index[max_idx]+pd.Timedelta(days=15), y_max-300000, f"Max: ${y_max:,.2f} \n ({basic.index[max_idx].strftime('%d/%m/%y')})", fontsize=10, weight="bold", color=color2)


plt.tight_layout()
plt.savefig('charts/basic/close_volume.png')
plt.show()
plt.close()

In [ ]:
## Close and Volume
fig, ax = plt.subplots(figsize=default_figsize)
font = {'fontname':'Times New Roman', 'size': 13, 'weight' : 'bold'}


ax.set_title('ETH/USDT napi záró árfolyama és kereskedések száma (db)', **font)
ax.set_xlabel('dátum', **font)
line1 = ax.plot(basic.index, basic.eth_num_trades, color7, label="ETH/USDT napi kereskedések száma (db)")
ax.set_ylabel('ETH/USDT napi kereskedések száma (db)', **font)
ax.yaxis.set_major_formatter("{x:,.2f}")
ax.tick_params(axis="y", labelcolor=color7)

ax2 = ax.twinx()
line2 = ax2.plot(basic.index, basic.eth_close, color6, label="ETH záró árfolyama ($)", alpha=0.8)
ax2.set_ylabel('ETH záró árfolyama ($)', **font)
ax2.yaxis.set_major_formatter("${x:,.2f}")
ax2.tick_params(axis="y", labelcolor=color6)


lines = line1 + line2
labels = [l.get_label() for l in lines]
ax.legend(lines, labels, loc=0)

ax2.spines['left'].set_color(color7)
ax2.spines['right'].set_color(color6)

##MIN MAX POINT for CLOSE
# Find the minimum and maximum values
y_min = basic['eth_num_trades'].min()
y_max = basic['eth_num_trades'].max()

# Find the indices of the minimum and maximum values
min_idx = basic['eth_num_trades'].argmin()
max_idx = basic['eth_num_trades'].argmax()

# Add dots for the minimum and maximum values
ax.scatter(basic.index[min_idx], y_min, color=color7, s=15, zorder=3)
ax.scatter(basic.index[max_idx], y_max, color=color7, s=15, zorder=3)

# Add text labels for the minimum and maximum values
ax.text(basic.index[min_idx]+pd.Timedelta(days=-25), y_min-200000, f"Min: {y_min:,.2f} ({basic.index[min_idx].strftime('%d/%m/%y')})", fontsize=10, weight="bold", color=color7)
ax.text(basic.index[max_idx]+pd.Timedelta(days=-200), y_max-200000, f"Max: {y_max:,.2f} \n ({basic.index[max_idx].strftime('%d/%m/%y')})", fontsize=10, weight="bold", color=color7)

##MIN MAX POINT for VOLUME
# Find the minimum and maximum values
y_min = basic['eth_close'].min()
y_max = basic['eth_close'].max()

# Find the indices of the minimum and maximum values
min_idx = basic['eth_close'].argmin()
max_idx = basic['eth_close'].argmax()

# Add dots for the minimum and maximum values
ax2.scatter(basic.index[min_idx], y_min, color=color6, s=15, zorder=3)
ax2.scatter(basic.index[max_idx], y_max, color=color6, s=15, zorder=3)

# Add text labels for the minimum and maximum values
ax2.text(basic.index[min_idx]-pd.Timedelta(days=10), y_min-250, f"Min: ${y_min:.2f} ({basic.index[min_idx].strftime('%d/%m/%y')})", fontsize=10, weight="bold", color=color6)
ax2.text(basic.index[max_idx]+pd.Timedelta(days=15), y_max-300, f"Max: ${y_max:,.2f} \n ({basic.index[max_idx].strftime('%d/%m/%y')})", fontsize=10, weight="bold", color=color6)

plt.tight_layout()
plt.savefig('charts/basic/num_trades.png')
plt.show()
plt.close()

# Section I.

- BTC, Fed, Nasdaq, S&P500, US Breakeven, US Treasury, Gold and Silver price

In [ ]:
section1 = df[['btc_close', 'btc_volume', 'btc_num_trades', 'fed_eff_funds_rate_close', 'nasdaq_close', 'real_estate_index_close', 'sp500_close', 'silver_value',
       'us_breake_inf_rate_5y', 'us_breake_inf_rate_10y', 'us_treasury_10yt2y_diff', 'us_treasury_10yt3m_diff', 'us_treasury_total_vol', 'gold_value', 'eth_close']]

In [ ]:
section1.info()

In [ ]:
section1.shape

In [ ]:
plt.figure(figsize=default_figsize)
sns.heatmap(np.log(section1).diff().dropna().corr(), annot=True, cmap="Blues", fmt=".2f");
plt.show()

In [ ]:
test = df[[i for i in df if i != "month"]]

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()

In [ ]:
test = pd.DataFrame(scaler.fit_transform(df.drop(columns="month")), columns=df.drop(columns="month").columns)

In [ ]:
X = test.drop(labels=['eth_close'], axis=1)
y = test["eth_close"]

In [ ]:
X_train = X.iloc[:1611, :]
y_train = y.iloc[:1611]

X_test = X.iloc[1611:, :]
y_test = y.iloc[1611:]


In [ ]:
X_test.shape

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Define your input data
# In this example, we have 3 features and 100 samples
X = X_train

# Define your output data
# In this example, we want to predict a single number for each sample
y = y_train

# Define your model architecture
model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(None, 39)))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(32, return_sequences=True))
model.add(Dense(1))

# Compile your model
model.compile(loss='mse', optimizer='adam')

# Train your model
model.fit(X.values.reshape(1611, 1, 39), y, epochs=20)

# Use your model to make predictions
predictions = model.predict(X_test.values.reshape(403,1,39))


In [ ]:
print(y_test)

In [ ]:
predictions = predictions.reshape(403,1)

In [ ]:
plt.plot(predictions)

In [ ]:
plt.plot(y_test)

### APPENDIX

### Plot shifted correlation between eth_volume and eth_num_trades

In [ ]:
is_stationary(np.log(basic.eth_volume).diff().dropna())

In [ ]:
is_stationary(np.log(basic.eth_close).diff().dropna())

In [ ]:
stationary_eth_close = np.log(basic.eth_close).diff().dropna()

In [ ]:
stationary_eth_volume = np.log(basic.eth_volume).diff().dropna()

In [ ]:
stationary_eth_num_trades = np.log(basic.eth_num_trades).diff().dropna()

In [ ]:
from scipy import signal
def ccf_values(series1, series2):
    p = series1
    q = series2
    p = (p - np.mean(p)) / (np.std(p) * len(p))
    q = (q - np.mean(q)) / (np.std(q))  
    c = np.correlate(p, q, 'full')
    return c
    
ccf_ielts = ccf_values(stationary_eth_close, stationary_eth_volume)
ccf_ielts

In [ ]:
lags = signal.correlation_lags(len(stationary_eth_close), len(stationary_eth_volume))

def ccf_plot(lags, ccf):
    fig, ax =plt.subplots(figsize=(9, 6))
    ax.plot(lags, ccf)
    ax.axhline(-2/np.sqrt(23), color='red', label='5% confidence interval')
    ax.axhline(2/np.sqrt(23), color='red')
    ax.axvline(x = 0, color = 'black', lw = 1)
    ax.axhline(y = 0, color = 'black', lw = 1)
    ax.axhline(y = np.max(ccf), color = color6, lw = 1,  linestyle='--', label = 'highest +/- correlation')
    ax.axhline(y = np.min(ccf), color = color6, lw = 1, 
    linestyle='--')
    ax.set(ylim = [-1, 1])
    ax.set_title('Cross Correation Eth Close and ETH Volume', weight='bold', fontsize = 15)
    ax.set_ylabel('Correlation Coefficients', weight='bold', fontsize = 12)
    ax.set_xlabel('Time Lags', weight='bold', fontsize = 12)
    plt.legend()
    
ccf_plot(lags, ccf_ielts)

In [ ]:
correlation = signal.correlate(stationary_eth_close, stationary_eth_volume, mode="full")
lags = signal.correlation_lags(stationary_eth_close.size, stationary_eth_volume.size, mode="full")
lag = lags[np.argmax(correlation)]
print("Most effective lag:", lag)
np.max(ccf_ielts), np.min(ccf_ielts)

In [ ]:
##Num trades

In [ ]:
ccf_ielts = ccf_values(stationary_eth_close, stationary_eth_num_trades)
lags = signal.correlation_lags(len(stationary_eth_close), len(stationary_eth_num_trades))
ccf_plot(lags, ccf_ielts)

In [ ]:
lag = lags[np.argmax(correlation)]

In [ ]:
lag

### Findings: 
It is a non-stationary dataset, with a heavy trend and seasonality based on 365 day period

In [ ]:
## Check for stationarity
from statsmodels.tsa.stattools import adfuller

def is_stationary(timeseries):
    """
    Determines if a time series is stationary using the Augmented Dickey-Fuller (ADF) test.

    Parameters:
    timeseries (pandas.Series): The time series to test for stationarity.
    significance_level (float): The significance level used to determine the critical value for the test.

    Returns:
    bool: True if the time series is stationary, False otherwise.
    """

    # Perform ADF test
    adf_result = adfuller(timeseries)

    # Get test statistic and critical values
    test_statistic = adf_result[0]
    critical_values = adf_result[4]
    print("Result of Dickey-Fuller test:")
    df_output = pd.Series(adf_result[0:4], index=["Test Statistic", "p-value", "#Lags Used", "Number of Observations Used"])
    
    for key, value in adf_result[4].items():
        df_output["Critical Value (%s)"%key] = value
    print(df_output)
        
        
    # Compare test statistic to critical values to determine stationarity
    if test_statistic < critical_values['5%']:  ## 5% significance level
        return True
    else:
        return False
    


In [ ]:
is_stationary(df.eth_close)

In [ ]:
df_log = np.log(df.eth_close)